# Generating a MIDI music file from a time series: a mostly useless but quite cool routine
*Rémy Lapere - eScience course - 8/11/2022*

The key library for that is miditime. Then, all you need to do is build an array specifying (time, pitch, velocity, duration) for every time step. The pitch is based on the values in the time series.

In this example we will compose the original music: *Evolution of annual mean temperature in the Arctic in C minor* from NorESM CMIP6 data in scenario ssp585.

Main steps:
- define the minimum and maximum pitch
- rescale the data to this range
- apply a key to obtain a nicer harmony
- manipulate the data to avoid repetitions and generate info on time, velocity and duration
- generate a MIDI file
- export that MIDI file to your favorite software

For a more comprehensive reference see *de Mora et al., 2020* - https://gc.copernicus.org/articles/3/263/2020/

In [1]:
import xarray as xr
xr.set_options(display_style='html')
import intake
import cftime
import numpy as np
import pandas as pd
from miditime.miditime import MIDITime

Define minimum and maximum notes for the song (middle C is 60, below 25 is very low, higher than 100 is very high). A unit corresponds to a semitone.

In [2]:
minpitch, maxpitch = 24, 96

Load data from pangeo (here surface temperature in historical and ssp585 experiments)

In [3]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
cat = col.search(variable_id=['tas'], member_id=['r1i1p1f1'], source_id=['NorESM2-LM'], experiment_id=['historical','ssp585'], table_id=['Amon'])

In [4]:
dset_dict = cat.to_dataset_dict(zarr_kwargs={'use_cftime':True})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [5]:
dataset_list = list(dset_dict.keys())

In [6]:
AR_hist = dset_dict[dataset_list[1]] # historical experiment
AR_fut5 = dset_dict[dataset_list[0]] # ssp585 experiment

Extract the area/period of interest, and rescale the time series to the range of authorized pitch

In [7]:
def ext_pole():
    """
    function for extraction area/period of interest
    and concatenate into a single numpy
    """
    lmi,lma = 60,90

    AR_hist_yr_ARC = AR_hist.sel(lat=slice(lmi,lma))
    AR_hist_yr_ARC = AR_hist_yr_ARC.reduce(np.mean,dim=('lat','lon'))
    AR_hist_yr_ARC = AR_hist_yr_ARC.resample(time='1Y').mean()
    histar = AR_hist_yr_ARC.tas.values

    arc585 = AR_fut5.sel(lat=slice(lmi,lma))
    arc585 = arc585.reduce(np.mean,dim=('lat','lon'))
    arc585 = arc585.resample(time='1Y').mean()
    arc585 = arc585.tas.values
    
    out585 = np.append(histar,arc585)
    return out585
    
n585 = ext_pole()

mini = np.min(n585)
maxi = np.max(n585)

# normalize data to [0,1]
n585_ = (n585-mini)/(maxi-mini)

# apply a mapping to the range of authorized min/max pitch
n585_ = minpitch+n585_*(maxpitch-minpitch)

# if you want to associate increasing temperature with lower pitch notes
n585_ = minpitch-n585_+maxpitch

# make it an integer type because MIDI only handles semitones
n585_ = n585_.astype(int)

# store notes into a data set along with time steps
df_ = pd.DataFrame({'val':n585_,'step':range(len(n585_))})

- So far we have a list of notes and time steps... but not all the notes can work together in harmony.

- In the next part we interpolate the notes to the nearest "authorized" note according to the chosen key.

- Here the example key is C minor, i.e. authorized notes are: C,Eb,G.

- Our 'zero' note is 24 which corresponds to C, 2 octaves (1 octave is 12 semitones) below middle C.

- Therefore the list of authorized notes is {0,3,7}mod(12)

In [8]:
def _to_chords_(df, key):
    """
    function to map the original notes to the defined key
    """
    notes = range(len(np.arange(minpitch,maxpitch+1,1)))
    notes_ = np.arange(minpitch,maxpitch+1,1)
    dom = np.mod(notes,12)==key[0]
    tir = np.mod(notes,12)==key[1]
    qui = np.mod(notes,12)==key[2]
    auth = dom+tir+qui
    auth_notes = notes_[auth]
    notin = df.val.values
    i=0
    for nn in notin:
        dist = np.abs(auth_notes-nn)
        tru_note = auth_notes[np.argmin(dist)]
        notin[i] = tru_note
        i=i+1
    df['val'] = notin
    return df

In [9]:
def extract_sdt(indata,kkeys):
    """
    function to aggregate consecutive notes
    and includes info on duration/velocity
    """
    didif = [indata['step'].values[0]]
    indata = _to_chords_(indata,kkeys)
    for i in np.arange(1,len(indata.val.values)):
        if indata['val'].values[i]==indata['val'].values[i-1]:
            didif = np.append(didif,indata['step'].values[i-1])
        else:
            didif = np.append(didif,indata['step'].values[i])
    indata['dif'] = didif
    steps = indata.groupby(['dif','val'],as_index=False).count().step.values
    vals = (indata.groupby(['dif','val'],as_index=False).mean().val.values).astype(int)
    new_df = pd.DataFrame({'note':vals,
                       'steps':np.cumsum(steps)-np.min(np.cumsum(steps)),
                       'duration':np.append(steps[1:],2),
                      'force':np.repeat(127,len(steps))})
    for j in np.arange(1,len(new_df.note.values)):
        if new_df.note.values[j] == new_df.note.values[j-1]:
            new_df.steps[j] = new_df.steps.values[j-1]
    dur = new_df.groupby(['steps'],as_index=False).sum().duration.values
    new_df = new_df.drop_duplicates(['steps','note'])
    new_df.duration = dur
    new_df['force'] = np.linspace(100,126,len(dur)).astype(int)
    new_df = new_df[['steps','note','force','duration']]
    return new_df

ext_mus = extract_sdt(df_,[0,3,7])
# save to csv
ext_mus.to_csv('arc_tas_music_585')

In [11]:
def to_midi(infile,nm):
    """
    function to convert the data to MIDI file
    """
    mymidi = MIDITime(160, nm+'.mid') # 160 is the tempo
    music = np.array(pd.read_csv(infile,skiprows=1,header=None,index_col=0)).tolist()
    # Add a track with those notes                                                                                                                                                                              
    mymidi.add_track(music)
    # Output the .mid file                                                                                                                                                                                      
    mymidi.save_midi()
    
# the exported MIDI file can be played with a dedicated audio player (e.g. GarageBand)
to_midi('arc_tas_music_585','arc_tas_music_585')

87 0 1 100
91 1 1 100
79 2 2 100
84 4 2 100
87 6 2 100
84 8 2 100
91 10 3 100
87 13 1 101
91 14 2 101
87 16 1 101
91 17 2 101
87 19 8 101
79 27 2 101
87 29 1 102
84 30 2 102
87 32 2 102
84 34 3 102
87 37 1 102
84 38 1 102
87 39 1 103
84 40 1 103
87 41 1 103
84 42 1 103
79 43 1 103
84 44 2 103
79 46 1 104
84 47 1 104
75 48 2 104
79 50 3 104
87 53 2 104
84 55 3 104
87 58 1 104
84 59 1 105
87 60 1 105
79 61 2 105
87 63 2 105
96 65 1 105
87 66 2 105
84 68 2 106
79 70 1 106
91 71 1 106
79 72 1 106
75 73 1 106
84 74 1 106
79 75 1 107
84 76 1 107
87 77 1 107
84 78 1 107
79 79 1 107
84 80 2 107
87 82 1 108
84 83 2 108
87 85 3 108
84 88 1 108
91 89 1 108
79 90 2 108
84 92 4 108
79 96 2 109
84 98 1 109
79 99 1 109
87 100 1 109
79 101 1 109
75 102 1 109
79 103 1 110
84 104 2 110
79 106 1 110
72 107 1 110
84 108 2 110
79 110 2 110
87 112 1 111
84 113 1 111
87 114 2 111
84 116 1 111
91 117 1 111
87 118 2 111
84 120 2 112
79 122 2 112
84 124 1 112
87 125 1 112
84 126 1 112
87 127 1 112
79 128 1 113
